In [2]:
import numpy as np
import subprocess
import os
import sys
import argparse
import glob
from astropy.io import fits as pyfits
import matplotlib.pyplot as plt
from urllib.request import urlopen
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.time import Time

In [3]:
fname = "VLASS_dyn_summary.php"

### Get Tiles Function

In [4]:
import urllib.request

url = 'https://archive-new.nrao.edu/vlass/VLASS_dyn_summary.php'
output_file = 'VLASS_dyn_summary.php'

urllib.request.urlretrieve(url, output_file)

print(f'File downloaded to: {output_file}')

File downloaded to: VLASS_dyn_summary.php


In [32]:
def get_tiles():
    """ Get tiles 
    I ran wget https://archive-new.nrao.edu/vlass/VLASS_dyn_summary.php
    """

    inputf = open(fname, "r")
    lines = inputf.readlines()
    inputf.close()

    header = list(filter(None, lines[0].split("  ")))
    # get rid of white spaces
    header = np.array([val.strip() for val in header])

    names = []
    dec_min = []
    dec_max = []
    ra_min = []
    ra_max = []
    obsdate = []
    epoch = []

    # Starting at lines[3], read in values
    for line in lines[3:]:
        dat = list(filter(None, line.split("  "))) 
        dat = np.array([val.strip() for val in dat]) 
        names.append(dat[0])
        dec_min.append(float(dat[1]))
        dec_max.append(float(dat[2]))
        ra_min.append(float(dat[3]))
        ra_max.append(float(dat[4]))
        obsdate.append(dat[6])
        epoch.append(dat[5])

    names = np.array(names)
    dec_min = np.array(dec_min)
    dec_max = np.array(dec_max)
    ra_min = np.array(ra_min)
    ra_max = np.array(ra_max)
    obsdate = np.array(obsdate)
    epoch = np.array(epoch)

    return (names, dec_min, dec_max, ra_min, ra_max, epoch, obsdate)


(array(['T01t01', 'T01t01', 'T01t01', ..., 'T32t02', 'T32t02', 'T32t02'],
       dtype='<U6'),
 array([-40., -40., -40., ...,  85.,  85.,  85.]),
 array([-36., -36., -36., ...,  90.,  90.,  90.]),
 array([ 0.,  0.,  0., ..., 12., 12., 12.]),
 array([ 0.5,  0.5,  0.5, ..., 24. , 24. , 24. ]),
 array(['VLASS2.1', 'VLASS3.1', 'VLASS1.1', ..., 'VLASS3.1', 'VLASS2.1',
        'VLASS1.1'], dtype='<U8'),
 array(['2020-10-25', '2023-06-06', '2018-02-07', ..., '2023-04-24',
        '2020-08-29', '2017-09-29'], dtype='<U13'))

### Search Tiles Function


In [6]:
ra_fungi = "16h43m48.201s"
dec_fungi = "+41d02m43.38s"
fungi1 = SkyCoord(ra = ra_fungi, dec = dec_fungi)

ra_mush = "250.950837548"
dec_mush = "+41.0453830323"
mushroom = SkyCoord(ra = ra_mush, dec = dec_mush, unit = "deg")


In [7]:
def search_tiles(tiles, c):
    """ Now that you've processed the file, search for the given RA and Dec
    
    Parameters
    ----------
    c: SkyCoord object
    """
    ra_h = c.ra.hour
    dec_d = c.dec.deg
    names, dec_min, dec_max, ra_min, ra_max, epochs, obsdate = tiles
    has_dec = np.logical_and(dec_d > dec_min, dec_d < dec_max)
    has_ra = np.logical_and(ra_h > ra_min, ra_h < ra_max)
    in_tile = np.logical_and(has_ra, has_dec)
    name = names[in_tile]
    epoch = epochs[in_tile]
    date = obsdate[in_tile]
    if len(name) == 0:
        print("Sorry, no tile found.")
        return None, None, None
    else:
        return name, epoch, date
    
search_tiles(get_tiles(), mushroom)

(array(['T21t17', 'T21t17', 'T21t17'], dtype='<U6'),
 array(['VLASS1.2', 'VLASS3.2', 'VLASS2.2'], dtype='<U8'),
 array(['2019-05-04', 'Scheduled', '2021-11-14'], dtype='<U13'))

### Get Subtiles Function

In [18]:
def get_subtiles(tilename, epoch):
    """ For a given tile name, get the filenames in the VLASS directory.
    Parse those filenames and return a list of subtile RA and Dec.
    RA and Dec returned as a SkyCoord object
    """
    if epoch =='VLASS1.2':
        epoch = 'VLASS1.2v2'
    elif epoch =='VLASS1.1':
        epoch = 'VLASS1.1v2'
    url_full = 'https://archive-new.nrao.edu/vlass/quicklook/%s/%s/' %(epoch,tilename)
    print(url_full)
    urlpath = urlopen(url_full)
    # Get site HTML coding
    string = (urlpath.read().decode('utf-8')).split("\n")
    # clean the HTML elements of trailing and leading whitespace
    vals = np.array([val.strip() for val in string])
    # Make list of HTML link elements
    keep_link = np.array(["href" in val.strip() for val in string])
    # Make list of HTML elements with the tile name
    keep_name = np.array([tilename in val.strip() for val in string])
    # Cross reference the two lists above to keep only the HTML elements with the tile name and a link
    string_keep = vals[np.logical_and(keep_link, keep_name)]
    # Keep only the links from the HTML elements (they are the 7th element since 6 quote marks precede it)
    fname = np.array([val.split("\"")[7] for val in string_keep])
    # Take out the element of the link that encodes the RA and declination
    pos_raw = np.array([val.split(".")[4] for val in fname])
    if '-' in pos_raw[0]:
        # dec < 0
        ra_raw = np.array([val.split("-")[0] for val in pos_raw])
        dec_raw = np.array([val.split("-")[1] for val in pos_raw])
    else:
        # dec > 0
        ra_raw = np.array([val.split("+")[0] for val in pos_raw])
        dec_raw = np.array([val.split("+")[1] for val in pos_raw])
    ra = []
    dec = []
    for ii,val in enumerate(ra_raw):
        # 24 hours is the same as hour 0
        if val[1:3] == '24':
            rah = '00'
        else:
            rah = val[1:3]
        # calculate RA in hours mins and seconds
        hms = "%sh%sm%ss" %(rah, val[3:5], val[5:])
        ra.append(hms)
        # calculate Dec in degrees arcminutes and arcseconds
        dms = "%sd%sm%ss" %(
                dec_raw[ii][0:2], dec_raw[ii][2:4], dec_raw[ii][4:])
        dec.append(dms)
    ra = np.array(ra)
    dec = np.array(dec)
    c_tiles = SkyCoord(ra, dec, frame='icrs')
    return fname, c_tiles

get_subtiles('T21t17','VLASS1.2')

## maybe look to see if we need to cut whitespace

https://archive-new.nrao.edu/vlass/quicklook/VLASS1.2v2/T21t17/


(array(['VLASS1.2.ql.T21t17.J160236+403000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160239+413000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160241+423000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160244+433000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160749+403000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160757+413000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160804+423000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J160812+433000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161303+403000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161315+413000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161327+423000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161340+433000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161816+403000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161833+413000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161850+423000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J161908+433000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J162329+403000.10.2048.v1/',
        'VLASS1.2.ql.T21t17.J16

### Get Cutout Function

In [19]:
def get_cutout(imname, name, c, epoch):
    print("Generating cutout")
    # Position of source
    ra_deg = c.ra.deg
    dec_deg = c.dec.deg
    print("Cutout centered at position %s,%s" %(ra_deg, dec_deg))

    # Open image and establish coordinate system
    im = pyfits.open(imname, ignore_missing_simple=True)[0].data[0,0]
    return im



In [ ]:
def get_cutout(imname, name, c, epoch):
    print("Generating cutout")
    # Position of source
    ra_deg = c.ra.deg
    dec_deg = c.dec.deg

    print("Cutout centered at position %s,%s" %(ra_deg, dec_deg))

    # Open image and establish coordinate system
    im = pyfits.open(imname, ignore_missing_simple=True)[0].data[0,0]
    w = WCS(imname)

    # Find the source position in pixels.
    # This will be the center of our image.
    src_pix = w.wcs_world2pix([[ra_deg, dec_deg, 0, 0]], 0)
    x = src_pix[0,0]
    y = src_pix[0,1]

    # Check if the source is actually in the image
    pix1 = pyfits.open(imname)[0].header['CRPIX1']
    pix2 = pyfits.open(imname)[0].header['CRPIX2']
    badx = np.logical_or(x < 0, x > 2 * pix1)
    bady = np.logical_or(y < 0, y > 2 * pix1)
    if np.logical_or(badx, bady):
        print("Tile has not been imaged at the position of the source")
        return None

    else:
        # Set the dimensions of the image
        # Say we want it to be 13.5 arcseconds on a side,
        # to match the DES images
        delt1 = pyfits.open(imname)[0].header['CDELT1']
        delt2 = pyfits.open(imname)[0].header['CDELT2']
        cutout_size = 12 / 3600 # in degrees
        dside1 = -cutout_size/2./delt1
        dside2 = cutout_size/2./delt2

        vmin = -1e-4
        vmax = 1e-3

        im_plot_raw = im[int(y-dside1):int(y+dside1),int(x-dside2):int(x+dside2)]
        im_plot = np.ma.masked_invalid(im_plot_raw)

        # 3-sigma clipping
        rms_temp = np.ma.std(im_plot)
        keep = np.ma.abs(im_plot) <= 3*rms_temp
        rms = np.ma.std(im_plot[keep])

        peak_flux = np.ma.max(im.flatten())

        plt.imshow(
                np.flipud(im_plot),
                extent=[-0.5*cutout_size*3600.,0.5*cutout_size*3600.,
                        -0.5*cutout_size*3600.,0.5*cutout_size*3600],
                vmin=vmin,vmax=vmax,cmap='YlOrRd')

        peakstr = "Peak Flux %s mJy" %(np.round(peak_flux*1e3, 3))
        rmsstr = "RMS Flux %s mJy" %(np.round(rms*1e3, 3))
        plt.title(name + ": %s; %s" %(peakstr, rmsstr))
        plt.xlabel("Offset in RA (arcsec)")
        plt.ylabel("Offset in Dec (arcsec)")

        plt.savefig(name + "_" + epoch + ".png") 
        plt.close()

        return peak_flux,rms

### Run Search Function

In [51]:
def run_search(name, c, date=None):
    """ 
    Searches the VLASS catalog for a source

    Parameters
    ----------
    names: name of the sources
    c: coordinates as SkyCoord object
    date: date in astropy Time format
    """
    print("Running for %s" %name)
    print("Coordinates %s" %c)
    print("Date %s" %date)

    # Find the VLASS tile(s)
    tiles = get_tiles()
    tilenames, epochs, obsdates = search_tiles(tiles, c)

    past_epochs = ["VLASS1.1v2", "VLASS1.2v2", "VLASS2.1", "VLASS2.2", "VLASS3.1"]
    current_epoch = "VLASS3.2"

    if tilenames[0] is None:
        print("There is no VLASS tile at this location")

    else:
        for ii,tilename in enumerate(tilenames):
            epoch = epochs[ii]
            obsdate = obsdates[ii]
            print(tilename)
            print(epoch)
            # Adjust name so it works with the version 2 ones for 1.1 and 1.2
            if epoch=='VLASS1.2':
                epoch = 'VLASS1.2v2'
            elif epoch =='VLASS1.1':
                epoch = 'VLASS1.1v2'

        
            if epoch not in past_epochs:
                if epoch == current_epoch:
                    # Make list of observed tiles 
                    url_full = 'https://archive-new.nrao.edu/vlass/quicklook/%s/' %(epoch)
                    print(url_full)
                    urlpath = urlopen(url_full)
                    # Get site HTML coding
                    string = (urlpath.read().decode('utf-8')).split("\n")
                    # clean the HTML elements of trailing and leading whitespace
                    vals = np.array([val.strip() for val in string])
                    # Make list of useful html elements
                    files = np.array(['alt="[DIR]"' in val.strip() for val in string])
                    useful = vals[files]
                    # Splice out the name from the link
                    obsname = np.array([val.split("\"")[7] for val in useful])
                    observed_current_epoch = np.char.replace(obsname, '/', '')

                    return obsname
                    
                else:
                    pass


            
                print("Sorry, tile will be observed in a later epoch")

    

run_search('T21t17', fungi1)


Running for T21t17
Coordinates <SkyCoord (ICRS): (ra, dec) in deg
    (250.9508375, 41.04538333)>
Date None
T21t17
VLASS1.2
T21t17
VLASS3.2
https://archive-new.nrao.edu/vlass/quicklook/VLASS3.2/


array(['T07t04', 'T07t14', 'T07t19', 'T07t25', 'T07t28', 'T07t35',
       'T08t04', 'T08t35', 'T09t13', 'T09t16', 'T09t19', 'T10t20',
       'T10t23', 'T10t26', 'T15t07', 'T15t09', 'T15t10', 'T15t12',
       'T15t13', 'T15t15', 'T15t16', 'T15t18', 'T15t25', 'T15t28',
       'T16t09', 'T16t11', 'T16t14', 'T16t16', 'T17t26', 'T17t29',
       'T17t32', 'T18t01', 'T18t03', 'T18t05', 'T18t07', 'T18t25',
       'T18t27', 'T18t29', 'T18t31', 'T19t12', 'T19t15', 'T19t25',
       'T19t28', 'T22t11', 'T22t13', 'T22t22', 'T22t24', 'T24t01',
       'T24t03', 'T24t04', 'T24t06', 'T24t08', 'T24t21', 'T24t23'],
      dtype='<U6')

In [ ]:
def run_search(name, c, date=None):
    """ 
    Searches the VLASS catalog for a source

    Parameters
    ----------
    names: name of the sources
    c: coordinates as SkyCoord object
    date: date in astropy Time format
    """
    print("Running for %s" %name)
    print("Coordinates %s" %c)
    print("Date %s" %date)

    # Find the VLASS tile(s)
    tiles = get_tiles()
    tilenames, epochs, obsdates = search_tiles(tiles, c)

    if tilenames[0] is None:
        print("There is no VLASS tile at this location")

    else:
        for ii,tilename in enumerate(tilenames):
            epoch = epochs[ii]
            # Adjust name so it works with the version 2 ones for 1.1 and 1.2
            if epoch=='VLASS1.2':
                epoch = 'VLASS1.2v2'
            elif epoch=='VLASS1.1':
                epoch = 'VLASS1.1v2'
            obsdate = obsdates[ii]
            if np.logical_and.reduce((epoch != "VLASS1.1", 
                epoch != "VLASS1.2",epoch != "VLASS2.1",epoch != 'VLASS2.2')):
                print("Sorry, tile will be observed in a later epoch")
            else:
                print("Tile found:")
                print(tilename, epoch)
                subtiles, c_tiles = get_subtiles(tilename, epoch)
                dist = c.separation(c_tiles)
                subtile = subtiles[np.argmin(dist)]
                print(subtile)

                url_get = "https://archive-new.nrao.edu/vlass/quicklook/%s/%s/%s" %(
                        epoch, tilename, subtile)
                imname="%s.I.iter1.image.pbcor.tt0.subim.fits" %subtile[0:-1]
                print(imname)
                if len(glob.glob(imname)) == 0:
                    fname = url_get + imname
                    cmd = "curl -O %s" %fname
                    print(cmd)
                    os.system(cmd)
                out = get_cutout(imname, name, c, epoch)
                if out is not None:
                    peak, rms = out
                    print("Peak flux is %s uJy" %(peak*1e6))
                    print("RMS is %s uJy" %(rms*1e6))
                    limit = rms*1e6
                    obsdate = Time(obsdate, format='iso').mjd
                    print("Tile observed on %s" %obsdate)
                    print(limit, obsdate)
                    #return limit,obsdate
    #return None

In [ ]:

if __name__=="__main__":
    parser = argparse.ArgumentParser(description=\
        '''
        Searches VLASS for a source.
        User needs to supply name, RA (in decimal degrees),
        Dec (in decimal degrees), and (optionally) date (in mjd).
        If there is a date, then will only return VLASS images taken after that date
        (useful for transients with known explosion dates).
        
        Usage: vlass_search.py <Name> <RA [deg]> <Dec [deg]> <(optional) Date [mjd]>
        ''', formatter_class=argparse.RawTextHelpFormatter)
        
    #Check if correct number of arguments are given
    if len(sys.argv) < 3:
        print("Usage: vlass_search.py <Name> <RA [deg]> <Dec [deg]> <(optional) Date [astropy Time]>")
        sys.exit()
     
    name = str(sys.argv[1])
    ra = float(sys.argv[2])
    dec = float(sys.argv[3])
    c = SkyCoord(ra, dec, unit='deg')

    if glob.glob("/Users/annaho/Dropbox/astro/tools/Query_VLASS/VLASS_dyn_summary.php"):
        pass
    else:
        print("Tile summary file is not here. Download it using wget:\
               wget https://archive-new.nrao.edu/vlass/VLASS_dyn_summary.php")

    if (len(sys.argv) > 4):
        date = Time(float(sys.argv[4]), format='mjd')
        print ('Searching for observations after %s' %date)
        run_search(name, c, date) 
    else:
        print ('Searching all obs dates')
        run_search(name, c) 
